## Data Exploration and Data Manpulation

In [1]:
import pandas as pd
inputFile='test_data_for_final.xlsx'
data=pd.read_excel(inputFile, sheet_name=None, index_col=0)
course=data['course']
slots=data['slots']
time_pref=data['time_pref']
day_pref=data['day_pref']
instructors=data['instructor']

### Input Tables

#### Input Table 1. Course information table. 
Course here refers to each course session, for example, DSO570 has two sessions, they will be treated as two seperate courses with the session numbers as the unique identifers.

Some courses need to be delivered on two different days. The "Parts" Column means how many sections a course has. Duration means how long a course section is expected to be. Num_students means how many students are expected to be registered (based on previous year data".

In [2]:
course.head()

,Parts,Duration,Num_students
Course,,,
16219,1,110,35
16215,1,110,35
16225,2,110,35
16280,2,80,45
16288,1,170,45


#### Input Table 2. Slot information table.

* We are dealing with department course scheduling problem. The slots table shows which slots are available. Originally, two slots can be assigned together, for example "MW Morning 8:00-9:50", we break up a combined assignment into two slots. The reason is that two combined slots maybe assigned to two different courses with only one secion. 

* Each slot has the information of the room, capacity, day, time period and the actual begin and end time. 

* A few 3 hour slots and 1.5 hour slots can be manually created in this table.

In [3]:
slots.head()

,Room,Capacity,Day,Time,Begin,End
Slot,,,,,,
1,ACC205,36,M,Early_Morning,08:00:00,09:50:00
2,ACC205,36,M,Morning,10:00:00,11:50:00
3,JFF 240,48,T,Evening,18:00:00,19:50:00
4,ACC205,36,W,Early_Morning,08:00:00,09:50:00
5,ACC205,36,M,Morning,10:00:00,11:50:00


####  Input Table 3. Instructor-course correspondence table
Some teachers teach multiple courses, as shown below.

In [4]:
instructors.head()

,Instructor
Course,
16219,"Pereira, Francis"
16215,"Pereira, Francis"
16225,"Majchrzak, Ann"
16280,"Selbe, Omeed"
16288,"Sosic, Greys"


#### Input Table 4 & 5 Time Preference Table and Day Preference Table
In the future, time preference table and day preference table data can be collected through a survey. 
The following two tables time_pref and day_pref are hypothetical data, assuming teachers preference towards different time of the day and the day of the week have been collected. If a teacher teach multiple courses,the preference scores towards day and time are the same for these courses.   

In [5]:
time_pref.head()

,Early Morning,Morning,Afternoon,Evening
Course,,,,
16219,1,2,2,1
16215,1,2,2,1
16225,0,2,2,1
16280,0,2,1,2
16288,1,1,2,1


In [6]:
day_pref.head()

,M,T,W,H,F
Course,,,,,
16219,2,1,2,1,0
16215,2,1,2,1,0
16225,1,2,1,2,1
16280,1,2,1,2,0
16288,1,2,1,2,1


### New Tables Created based on Input Data

#### Table 6. Preference scores towards time of each slot

Based on the day_pref and time_pref data, we can create a new data_frame that shows the preference scores for each course each day_time combintation. For example, the teacher teaching "16219" scores 1, 2, 2, 1 on early morning, morning, afternoon, and evening, and scores 2, 1, 2, 1,0 on M through F. By multipling the perference towards a day and the preference score on a time, we get the combined score. Any 0 zero score will result in 0 in the multiplication. The pref_df table is the result table.

In [7]:
pref=[]
for index, row in day_pref.iterrows():
    pref_row=[]
    for day in row.index:
        pref_row.extend(row.loc[day]*time_pref.loc[index,:].values)
    pref.append(pref_row)

col_names=[]
for day in "MTWHF":
    for time in ["Early_Morning", "Morning", "Afternoon", "Evening"]:
        col_names.append(day+"_"+time)

pref_df=pd.DataFrame(pref, index=day_pref.index, columns=col_names)     
pref_df.head()

,M_Early_Morning,M_Morning,M_Afternoon,M_Evening,T_Early_Morning,T_Morning,T_Afternoon,T_Evening,W_Early_Morning,W_Morning,W_Afternoon,W_Evening,H_Early_Morning,H_Morning,H_Afternoon,H_Evening,F_Early_Morning,F_Morning,F_Afternoon,F_Evening
Course,,,,,,,,,,,,,,,,,,,,
16219,2,4,4,2,1,2,2,1,2,4,4,2,1,2,2,1,0,0,0,0
16215,2,4,4,2,1,2,2,1,2,4,4,2,1,2,2,1,0,0,0,0
16225,0,2,2,1,0,4,4,2,0,2,2,1,0,4,4,2,0,2,2,1
16280,0,2,1,2,0,4,2,4,0,2,1,2,0,4,2,4,0,0,0,0
16288,1,1,2,1,2,2,4,2,1,1,2,1,2,2,4,2,1,1,2,1


#### Table 7. Manipulated Slots Information Table
* Manipulate the slots table to create a day_time combination variable and duation variable for later analysis.

In [8]:
slots['day_time']=slots.Day.str.cat(slots.Time, sep="_")

In [9]:
##The function to manipulate time data, convert it to intergers
import numpy as np
def convert(inputTime):
    try:
        hh, mm, ss=inputTime.split(':')
        ans=int(hh)*60+int(mm)+int(ss)/3600
    except:
        ans=np.nan
    return ans

##test
print(convert("9:50:00"))

590.0


In [10]:
duration=[]
for index, row in slots.iterrows():
    beg=convert(str(row['Begin']))
    end=convert(str(row['End']))
    duration.append(end-beg)
slots["Duration"]=duration

In [11]:
###This is the new slots table
slots.head()

,Room,Capacity,Day,Time,Begin,End,day_time,Duration
Slot,,,,,,,,
1,ACC205,36,M,Early_Morning,08:00:00,09:50:00,M_Early_Morning,110.0
2,ACC205,36,M,Morning,10:00:00,11:50:00,M_Morning,110.0
3,JFF 240,48,T,Evening,18:00:00,19:50:00,T_Evening,110.0
4,ACC205,36,W,Early_Morning,08:00:00,09:50:00,W_Early_Morning,110.0
5,ACC205,36,M,Morning,10:00:00,11:50:00,M_Morning,110.0


#### Table 8. Preference score table for optimization

In our test data, there are only 13 slots and 8 courses (12 sessions in total) in this dataset. In order to do optimization, we need to create a preference score table for each course and slot, based on the pref_df table and slots table. Simply speaking, we can access the day_time of a slot, and get the associated preference score from the pref_df table. Pref2_df is the result table.


In [12]:
pref2=[]
for ix1, row1 in pref_df.iterrows():
    scores=[]
    for ix2, row2 in slots.iterrows():
        score=row1.loc[row2.loc['day_time']]
        scores.append(score)
    pref2.append(scores)

In [13]:
pref2_df=pd.DataFrame(pref2, index=pref_df.index, columns=slots.index)    
pref2_df.head()

Slot,1,2,3,4,5,6,7,8,9,10,11,12,13
Course,,,,,,,,,,,,,
16219,2,4,1,2,4,2,2,4,2,2,2,2,2
16215,2,4,1,2,4,2,2,4,2,2,2,2,2
16225,0,2,2,0,2,1,4,2,4,4,4,4,1
16280,0,2,4,0,2,2,2,1,2,2,2,4,2
16288,1,1,2,1,1,1,4,2,4,4,4,2,1


## Optimization

In [14]:
courses_ix=course.index
slots_ix=slots.index

In [15]:
import gurobipy as grb
mod=grb.Model()

In [16]:
###define the variables, x[i,j] denotes whether assign course i with slot j
x={}
for i in courses_ix:
    for j in slots_ix:
        x[i,j]=mod.addVar(vtype=grb.GRB.BINARY, name="x[{0},{1}]".format(i, j))

In [17]:
### objective function
mod.setObjective(sum(x[i,j]*pref2_df.loc[i,j]
                     -x[i,j]*(slots.loc[j, "Duration"]-course.loc[i, "Duration"]/10) 
                     -x[i,j]*(slots.loc[j, "Capacity"]-course.loc[i, "Num_students"]/20) 
                     for i in courses_ix for j in slots_ix), sense=grb.GRB.MAXIMIZE)

In [18]:
### the number of slots assigned to a course should be the same as the number of times students meet
parts_constr={}
for i in courses_ix:
    parts_constr[i]=mod.addConstr(sum(x[i,j] for j in slots_ix)==course.loc[i,"Parts"])

In [19]:
##each slot can only be assigned to 1 course
slots_constr={}
for j in slots_ix:
    slots_constr[j]=mod.addConstr(sum(x[i,j] for i in courses_ix)<=1)

In [20]:
###XXX the duration of a course can not be greater than the duration of the slot
duration_constr={}
for i in courses_ix:
    for j in slots_ix:
        duration_constr[i]=mod.addConstr(slots.loc[j, "Duration"]-x[i,j]*course.loc[i, "Duration"]>=0)

In [21]:
## the number of students can not exceed the room capacity
students_constr={}
for i in courses_ix:
    for j in slots_ix:
        students_constr[i]=mod.addConstr(slots.loc[j, "Capacity"]-x[i,j]*course.loc[i, "Num_students"]>=0)

In [22]:
### if preference score=0, xij=0
pref_constr={}
for i in courses_ix:
    for j in slots_ix:
        if pref2_df.loc[i,j]==0:
            pref_constr[i,j]=mod.addConstr(x[i,j]==0)

In [23]:
mod.optimize()

Optimize a model with 237 rows, 104 columns and 424 nonzeros
Variable types: 0 continuous, 104 integer (104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [9e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -1850.000000
Presolve removed 221 rows and 55 columns
Presolve time: 0.00s
Presolved: 16 rows, 49 columns, 98 nonzeros
Variable types: 0 continuous, 49 integer (49 binary)

Root relaxation: objective -1.843000e+03, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    -1843.000000 -1843.0000  0.00%     -    0s

Explored 0 nodes (18 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: -1843 -1850 

Optimal solution found (tolerance 1.00e-04)
Best objective -1.843000000000e+03, 

### Compute metrics

### 1. Compute current satisfaction score

* Step 1. Produce the output table---scheduleing result

In [24]:
###Create an allocation table to show the allocation results
table=[]
for i in courses_ix:
    s=[]
    for j in slots_ix:  
        if(x[i,j].x==1):
            s.append(1)
        else: s.append(0)
    table.append(s)

In [25]:
allocation=pd.DataFrame(table, index=courses_ix, columns=slots_ix)
allocation.head()

Slot,1,2,3,4,5,6,7,8,9,10,11,12,13
Course,,,,,,,,,,,,,
16219,0,1,0,0,0,0,0,0,0,0,0,0,0
16215,0,0,0,0,1,0,0,0,0,0,0,0,0
16225,0,0,0,0,0,0,0,1,0,1,0,0,0
16280,0,0,1,0,0,0,0,0,0,0,0,1,0
16288,0,0,0,0,0,0,0,0,1,0,0,0,0


* Step 2: Compute the satisfaction score towards each scheduling decision.
Based on the pref2_df table, we can see the highest possible score a faculty towords an allocation is 4. After optimization, we obtained the allocation result, see the above allocation table. The following code compute the actual score faculty have towards each course scheduling result. If a course has two parts, say one receives 2 and one receives 4, then the preference score should be averaged, i.e., 3. The following two blocks of code compute the average satisfaction score.

In [26]:
pref_scores=[]
for i in courses_ix:
    s=[]
    for j in slots_ix:
        s.append(allocation.loc[i,j]*pref2_df.loc[i,j])
    pref_scores.append(s)
pref_scores


[[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 2, 0, 4, 0, 0, 0],
 [0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 0],
 [1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]

* Step 3. Compute the average satisfaction score as the metric

In [27]:
###compute average satisfaction score
course_scores=[]
for ss in pref_scores:
    i=0
    for s in ss:
        if s>0: i+=1
    course_scores.append(sum(ss)/i)
sum(course_scores)/len(course_scores)

3.1875

* Based on the above computation, we find the average preference score faculty towards course scheduling is 3 out of 4. Qualitatively, it is a relatively good result.

#### Compute time waste
* The following code compute on average time waste per course. On average, each course wasted 16 min of room usage. Because we only created 1 1.5 hour slot, that means some 80 min courses are assigned with 2 hour slots. In the actual optimation problem, we can create more 1.5 hour slots based on estimated number of 80 min courses, so less time will be wasted.

In [28]:
dur=[]
for i in courses_ix:
    s=[]
    for j in slots_ix:
        s.append(allocation.loc[i,j]*(slots.loc[j, "Duration"]-course.loc[i, "Duration"]))          
    dur.append(s)
dur

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 30.0, 0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 30.0, 0.0, 0.0, 0.0, 0.0, 30.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 10.0]]

In [29]:
time_waste=[]
for tw in dur:
    time_waste.append(sum(tw))
sum(time_waste)/len(time_waste)

16.25

#### Compute seat waste
* The following code compute on average seat waste per course. On average, each course wasted 13 seats in this example. This is probably due to two rooms have 76 seats but the expected number of students for courses are under 50. When we apply this optimization model on a whole dataset in the future, we expecte the seat waste will be much smaller. The following code can be ued to compute the seat waste metric.

In [30]:
cap=[]
for i in courses_ix:
    s=[]
    for j in slots_ix:
        s.append(allocation.loc[i,j]*(slots.loc[j, "Capacity"]-course.loc[i, "Num_students"]))          
    cap.append(s)
cap

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 13, 0, 13, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 28, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 28, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11]]

In [31]:
seat_waste=[]
for sw in cap:
    seat_waste.append(sum(sw))
sum(seat_waste)/len(seat_waste)

13.25

### Save the allocation result

* The following code create the allocation result table joined the slot information table, so we can view which room, day and time is assigned to each course.

In [32]:
table2=[]
for j in slots_ix:
    s=[]
    for i in courses_ix:
        if(x[i,j].x==1):
            s.append(1)
        else: s.append("")
    table2.append(s)
        

In [33]:
allocation2=pd.DataFrame(table2, index=slots_ix, columns=courses_ix)

In [34]:
slots2=slots.join(allocation2)

In [35]:
slots2.head()

,Room,Capacity,Day,Time,Begin,End,day_time,Duration,16219,16215,16225,16280,16288,16530,16278,16271
Slot,,,,,,,,,,,,,,,,
1,ACC205,36,M,Early_Morning,08:00:00,09:50:00,M_Early_Morning,110.0,,,,,,,1,
2,ACC205,36,M,Morning,10:00:00,11:50:00,M_Morning,110.0,1,,,,,,,
3,JFF 240,48,T,Evening,18:00:00,19:50:00,T_Evening,110.0,,,,1,,,,
4,ACC205,36,W,Early_Morning,08:00:00,09:50:00,W_Early_Morning,110.0,,,,,,,1,
5,ACC205,36,M,Morning,10:00:00,11:50:00,M_Morning,110.0,,1,,,,,,


In [37]:
slots2.to_excel("allocation_result.xlsx")